In [9]:
from ortools.linear_solver import pywraplp

### 1. Food manufacture 1

#### Entitities

- $V \in 1, \ldots, 5$: the types of oil. $v_1$ = VEG1, $v_2$ = VEG2, $v_5$ = OIL3.
- $M \in 1, \ldots, 6$: the six months in the problem.

#### Parameters

- $P_{vm}$: price to buy oil $v$ in month $m$.

- $H_v$: hardness of each oil $v$.

#### Decision variables

- $x_{vm}$: amount of each oil to _buy_ in each month.

- $u_{vm}$: amount of each oil to _refine_ in each month.

- $n_{vm}$: amount of each oil that is in storage at the end of each month.

- $y_m$: amount of product we make each month.

#### Objective function

$$ \max 150 \sum_{m \in M} y_m - 5 \sum_{m \in M, v \in V} n_{vm} - \sum_{v \in V, m \in M} P_{vm}x_{vm}$$

#### Constraints

subject to:

1. $$n_{vm} = n_{vm-1} + x_{vm} - u_{vm} \, \, \forall v \in V, m \in 2,\ldots, 6$$

2. $$n_{v1} = 500 \, \, \forall v \in V$$

3. $$n_{v6} >= 500 \, \,  \forall v \in V$$

4. $$n_{vm} \leq 1000 \, \, \forall v \in V, m \in M$$

5. $$y_m = \sum_{v \in V} u_{vm} \, \, \forall m \in M$$

6. $$3y_m \leq \sum_{v \in V} H_v u_{vm} \, \, \forall m \in M$$

7. $$6y_m \geq \sum_{v \in V} H_v u_{vm} \, \, \forall m \in M $$ 

8. $$u_{1m} + u_{2m} \leq 200 \, \, \forall m \in M$$

9. $$u_{3m} + u_{4m} + u_{5m} \leq 250 \, \, \forall m \in M$$

In [10]:
M = range(1, 7) # months
V = range(1, 6) # types of oil

P = {1:{1:110, 2:130, 3:110, 4:120, 5:100, 6:90},
     2:{1:120, 2:130, 3:140, 4:110, 5:120, 6:100},
     3:{1:130, 2:110, 3:130, 4:120, 5:150, 6:140},
     4:{1:110, 2:90, 3:100, 4:120, 5:110, 6:80},
     5:{1:115, 2:115, 3:95, 4:125, 5:105, 6:135}} # purchase price

H = {1:8.8, 2:6.1, 3:2.0, 4:4.2, 5:5.0}

In [11]:
## Setup solver
solver = pywraplp.Solver("FoodManufacture1", 
    pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
inf = solver.infinity()

## Decision variables
x_VM = {v: {m: solver.NumVar(lb=0, ub=inf, name = f"x_{v}_{m}") 
            for m in M} for v in V}

u_VM = {v: {m: solver.NumVar(lb=0, ub=250, name = f"u_{v}_{m}") 
            for m in M} for v in V}

n_VM = {v: {m: solver.NumVar(lb=0, ub=1000, name = f"n_{v}_{m}") 
            for m in M} for v in V}

y_M = {m: solver.NumVar(lb=0, ub=inf, name = f"y_{m}") for m in M}

## Objective function
solver.Maximize((150 * solver.Sum([y_M[m] for m in M])) # sales
                - (5 * solver.Sum([n_VM[v][m] for v in V for m in M])) # storage
                - solver.Sum([P[v][m] * x_VM[v][m] for v in V for m in M])) # buying

## Constraints

# continuity constraints
for v in V:
    solver.Add(n_VM[v][1] == 500 + x_VM[v][1] - u_VM[v][1])
for m in M[1:]:
    for v in V:
        solver.Add(n_VM[v][m] == n_VM[v][m-1] + x_VM[v][m] - u_VM[v][m])
    
# must have enough units at the end
for v in V:
    solver.Add(n_VM[v][6] >= 500)
    
# storage capacity limits
for v in V:
    for m in M:
        solver.Add(n_VM[v][m] <= 1000)
        
# final amount of product
for m in M:
    solver.Add(y_M[m] == solver.Sum([u_VM[v][m] for v in V]))
               
# hardness limits
for m in M:
    solver.Add(3 * y_M[m] <= solver.Sum([H[v] * u_VM[v][m] for v in V]))
    solver.Add(6 * y_M[m] >= solver.Sum([H[v] * u_VM[v][m] for v in V]))

# capacity limits
for m in M:
    solver.Add(u_VM[1][m] + u_VM[2][m] <= 200)
    solver.Add(u_VM[3][m] + u_VM[4][m] + u_VM[5][m] <= 250)

In [12]:
solver.Solve()
obj = solver.Objective().Value()
print(f"Solution found. Optimal policy generates £{int(round(obj, 0))} of profit.")

Solution found. Optimal policy generates £107843 of profit.


### 2. Food manufacture 2

As before, but with the addition of a new variable , which takes value 1 if an oil is used, 0 otherwise. 

#### Extra decision variables

- $\delta_{vm} \in \{ 0, 1\}$

#### Extra constraints

10. $$\sum_{v \in V} \delta_{vm} \leq 3 \, \, \forall m \in M$$

11. $$\delta_{m5} \geq \delta_{m1} \, \, \forall m \in M$$

12. $$\delta_{m5} \geq \delta_{m2} \, \, \forall m \in M$$

13. $$u_{vm} \geq 20 \delta_{vm} \, \, \forall m \in M$$

14. $$u_{vm} \leq 250 \delta_{vm} \, \, \forall m \in M$$



In [13]:
M = range(1, 7) # months
V = range(1, 6) # types of oil

P = {1:{1:110, 2:130, 3:110, 4:120, 5:100, 6:90},
     2:{1:120, 2:130, 3:140, 4:110, 5:120, 6:100},
     3:{1:130, 2:110, 3:130, 4:120, 5:150, 6:140},
     4:{1:110, 2:90, 3:100, 4:120, 5:110, 6:80},
     5:{1:115, 2:115, 3:95, 4:125, 5:105, 6:135}} # purchase price

H = {1:8.8, 2:6.1, 3:2.0, 4:4.2, 5:5.0}

In [14]:
## Setup solver
solver = pywraplp.Solver("FoodManufacture2", 
    pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
inf = solver.infinity()

## Decision variables
x_VM = {v: {m: solver.NumVar(lb=0, ub=inf, name = f"x_{v}_{m}") 
            for m in M} for v in V}

u_VM = {v: {m: solver.NumVar(lb=0, ub=250, name = f"u_{v}_{m}") 
            for m in M} for v in V}

n_VM = {v: {m: solver.NumVar(lb=0, ub=1000, name = f"n_{v}_{m}") 
            for m in M} for v in V}

y_M = {m: solver.NumVar(lb=0, ub=inf, name = f"y_{m}") for m in M}

delta_VM = {v: {m: solver.IntVar(lb=0, ub=1, name = f"delta_{v}_{m}")
                for m in M} for v in V}

## Objective function
solver.Maximize((150 * solver.Sum([y_M[m] for m in M])) # sales
                - (5 * solver.Sum([n_VM[v][m] for v in V for m in M])) # storage
                - solver.Sum([P[v][m] * x_VM[v][m] for v in V for m in M])) # buying

## Constraints

# continuity constraints
for v in V:
    solver.Add(n_VM[v][1] == 500 + x_VM[v][1] - u_VM[v][1])
for m in M[1:]:
    for v in V:
        solver.Add(n_VM[v][m] == n_VM[v][m-1] + x_VM[v][m] - u_VM[v][m])
    
# must have enough units at the end
for v in V:
    solver.Add(n_VM[v][6] >= 500)
    
# storage capacity limits
for v in V:
    for m in M:
        solver.Add(n_VM[v][m] <= 1000)
        
# final amount of product
for m in M:
    solver.Add(y_M[m] == solver.Sum([u_VM[v][m] for v in V]))
               
# hardness limits
for m in M:
    solver.Add(3 * y_M[m] <= solver.Sum([H[v] * u_VM[v][m] for v in V]))
    solver.Add(6 * y_M[m] >= solver.Sum([H[v] * u_VM[v][m] for v in V]))

# capacity limits
for m in M:
    solver.Add(u_VM[1][m] + u_VM[2][m] <= 200)
    solver.Add(u_VM[3][m] + u_VM[4][m] + u_VM[5][m] <= 250)
    
# Cannot use more than 3 types of oil
for m in M:
    solver.Add(solver.Sum([delta_VM[v][m] for v in V]) <= 3)

# If we use v1 or v2, we must use o3
for m in M:
    solver.Add(delta_VM[5][m] >= delta_VM[1][m])
    solver.Add(delta_VM[5][m] >= delta_VM[2][m])
    
# If we use an oil, we must use 20+ tons
for m in M:
    for v in V:
        solver.Add(u_VM[v][m] >= 20 * delta_VM[v][m])
        
# If delta=zero, used=0
for m in M:
    for v in V:
        solver.Add(u_VM[v][m] <= 251 * delta_VM[v][m])

In [15]:
solver.Solve()
obj = solver.Objective().Value()
print(f"Solution found. Optimal policy generates £{int(round(obj, 0))} of profit.")

Solution found. Optimal policy generates £100279 of profit.
